In [ ]:
from langchain_community.vectorstores import Chroma
import chromadb
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

In [ ]:
df = pd.read_csv('./data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv')
df

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter


loader = CSVLoader(file_path='./data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv', csv_args={
    'delimiter': ',',
    #'fieldnames': ['content', 'url', 'title', 'date', 'author', 'domain']
})

raw_documents = loader.load()


text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

print(raw_documents[0])
 
print("----------------------")
 
print(documents[0])

In [ ]:
from langchain_community.embeddings import FakeEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

chroma_client = chromadb.HttpClient(host='localhost', port=8000)
chroma_client.reset()

langchain_chroma = Chroma(
    client=chroma_client,
    collection_name="my_langchain_collection",
    embedding_function=embeddings,
)

langchain_chroma.add_documents(documents=documents)

In [ ]:
docs = langchain_chroma.similarity_search("What is the answer to global warming?")
print(docs[0].page_content)

In [ ]:
langchain_collection = chroma_client.get_collection("my_langchain_collection")

langchain_collection.query(
    query_embeddings=[i for i in range(768)],
    n_results=1,
)